# <span style="color:red">Random forest para imagenes promedio</span>

In [1]:
from sklearn.ensemble import RandomForestClassifier 
import cv2 as cv
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

/home/franklinsierra/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
path_classes = "/home/franklinsierra/Documentos/Biomedica/Polipos/notebooks/mix_avg_img/"
classes_names = os.listdir(path_classes)
print (classes_names)
dic_img ={}
#numero de arboles para el random forest
num_trees = 100
#bins para el histograma
bins = 8
test_size = 0.2
#semilla para reproducir los mismos resultados
seed = 9

for i, name_class in enumerate(classes_names):
    dic_img[classes_names[i]] = np.array(os.listdir(path_classes+classes_names[i]))
    print ("class: ", name_class, "total ", len(dic_img[classes_names[i]]))

['adenomaWl', 'hyperplasWl', 'hyperplasNbi', 'serratedWl', 'serratedNbi', 'adenomaNbi']
class:  adenomaWl total  40
class:  hyperplasWl total  21
class:  hyperplasNbi total  21
class:  serratedWl total  15
class:  serratedNbi total  15
class:  adenomaNbi total  40


In [3]:
predictor = []
target = []
resize_value = 100
for i, classes in enumerate(classes_names):
    print((i+1), "clase: ", classes, " target: ", (i+1), " total: ", len(dic_img[classes]))
    for img_name in dic_img[classes]:
        path_image = path_classes+classes+'/'+img_name
        img = cv.resize(cv.imread(path_image,0),(100,100))*1.
        flatt_img = img.reshape(resize_value*resize_value)        
        predictor.append(flatt_img)
        target.append(i+1)
predictor = np.r_[predictor]        
target = np.r_[target]

1 clase:  adenomaWl  target:  1  total:  40
2 clase:  hyperplasWl  target:  2  total:  21
3 clase:  hyperplasNbi  target:  3  total:  21
4 clase:  serratedWl  target:  4  total:  15
5 clase:  serratedNbi  target:  5  total:  15
6 clase:  adenomaNbi  target:  6  total:  40


### Normalizando 

In [4]:
predictor_norm = preprocessing.normalize(predictor)

In [5]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(predictor_norm,target,test_size=test_size)
print (y_test)

[6 1 4 5 4 6 2 3 6 2 2 2 3 2 6 6 1 1 6 3 1 3 1 6 1 1 6 1 5 3 3]


In [6]:
forest = RandomForestClassifier(n_jobs=2, oob_score=True, n_estimators=num_trees)
forest.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [36]:
forest.oob_decision_function_

array([[0.08333333, 0.25      , 0.16666667, 0.11111111, 0.22222222,
        0.16666667],
       [0.08571429, 0.4       , 0.31428571, 0.05714286, 0.11428571,
        0.02857143],
       [0.06666667, 0.13333333, 0.03333333, 0.        , 0.63333333,
        0.13333333],
       [0.08823529, 0.38235294, 0.02941176, 0.05882353, 0.35294118,
        0.08823529],
       [0.15      , 0.225     , 0.15      , 0.05      , 0.35      ,
        0.075     ],
       [0.        , 0.25714286, 0.02857143, 0.34285714, 0.34285714,
        0.02857143],
       [0.05405405, 0.27027027, 0.        , 0.2972973 , 0.        ,
        0.37837838],
       [0.        , 0.5       , 0.        , 0.07692308, 0.38461538,
        0.03846154],
       [0.13793103, 0.27586207, 0.        , 0.13793103, 0.27586207,
        0.17241379],
       [0.07894737, 0.18421053, 0.18421053, 0.26315789, 0.21052632,
        0.07894737],
       [0.025     , 0.175     , 0.075     , 0.15      , 0.425     ,
        0.15      ],
       [0.02631579, 0

In [7]:
forest.score(X_test,y_test)

0.16129032258064516

### Standardization

In [8]:
predictor_cent = preprocessing.scale(predictor)

In [10]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(predictor_cent,target,test_size=test_size)
print (y_test)

[3 6 1 6 1 1 2 2 6 4 3 6 6 1 1 1 4 3 6 6 5 1 6 5 6 6 6 1 3 1 5]


In [17]:
forest2 = RandomForestClassifier(n_jobs=2, oob_score=True, n_estimators=num_trees)
forest2.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [18]:
forest2.score(X_test,y_test)

0.22580645161290322

# LOOCV

### Normalized

In [14]:
cv1 = KFold(n_splits = predictor_norm.shape[0], shuffle = True, random_state = 1)

In [16]:
acc = np.mean(cross_val_score(forest, predictor_norm, target, scoring = "accuracy", cv = cv1, n_jobs=1))
acc

0.18421052631578946

### Standardization

In [19]:
cv2 = KFold(n_splits = predictor_cent.shape[0], shuffle = True, random_state = 1)

In [22]:
acc2 = np.mean(cross_val_score(forest2, predictor_norm, target, scoring = "accuracy", cv = cv2, n_jobs=1))
acc2

0.17763157894736842

# <span style="color:red">Random forest para varianza</span>

In [24]:
path_classes2 = "/home/franklinsierra/Documentos/Biomedica/Polipos/notebooks/mix_var_img/"
classes_names2 = os.listdir(path_classes2)
print (classes_names)
dic_img ={}
#numero de arboles para el random forest
num_trees = 100
#bins para el histograma
bins = 8
test_size = 0.2
#semilla para reproducir los mismos resultados
seed = 9

for i, name_class in enumerate(classes_names2):
    dic_img[classes_names2[i]] = np.array(os.listdir(path_classes2+classes_names2[i]))
    print ("class: ", name_class, "total ", len(dic_img[classes_names2[i]]))

['adenomaWl', 'hyperplasWl', 'hyperplasNbi', 'serratedWl', 'serratedNbi', 'adenomaNbi']
class:  hiperplasNbi total  21
class:  adenomaWl total  40
class:  serratedWl total  15
class:  serratedNbi total  15
class:  hiperplasWl total  21
class:  adenomaNbi total  40


In [25]:
predictor2 = []
target2 = []
resize_value = 100
for i, classes in enumerate(classes_names2):
    print((i+1), "clase: ", classes, " target: ", (i+1), " total: ", len(dic_img[classes]))
    for img_name in dic_img[classes]:
        path_image = path_classes2+classes+'/'+img_name
        img = cv.resize(cv.imread(path_image,0),(100,100))*1.
        flatt_img = img.reshape(resize_value*resize_value)        
        predictor2.append(flatt_img)
        target2.append(i+1)
predictor2 = np.r_[predictor2]        
target2 = np.r_[target2]

1 clase:  hiperplasNbi  target:  1  total:  21
2 clase:  adenomaWl  target:  2  total:  40
3 clase:  serratedWl  target:  3  total:  15
4 clase:  serratedNbi  target:  4  total:  15
5 clase:  hiperplasWl  target:  5  total:  21
6 clase:  adenomaNbi  target:  6  total:  40


### Normalizando 

In [26]:
predictor2_norm = preprocessing.normalize(predictor2)

In [27]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train2, X_test2, y_train2, y_test2 = train_test_split(predictor2_norm,target2,test_size=test_size)
print (y_test2)

[2 4 1 2 2 4 6 4 5 2 2 2 5 6 6 6 5 4 5 4 6 2 3 6 2 5 2 1 2 1 6]


In [35]:
forest3 = RandomForestClassifier(n_jobs=2, oob_score=True, n_estimators=num_trees)
forest3.fit(X_train2,y_train2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [36]:
forest3.score(X_test2,y_test2)

0.1935483870967742

### Standardization

In [30]:
predictor2_cent = preprocessing.scale(predictor2)

In [32]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(predictor2_cent,target2,test_size=test_size)
print (y2_test)

[6 2 2 6 2 5 2 2 6 1 6 6 2 2 2 1 2 5 2 5 1 6 6 6 6 6 6 6 2 2 3]


In [37]:
forest4 = RandomForestClassifier(n_jobs=2, oob_score=True, n_estimators=num_trees)
forest4.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [38]:
forest4.score(X_test, y_test)

0.4838709677419355

# LOOCV

### Normalizado

In [39]:
cv3 = KFold(n_splits = predictor2_norm.shape[0], shuffle = True, random_state = 1)

In [40]:
acc3 = np.mean(cross_val_score(forest, predictor2_norm, target2, scoring = "accuracy", cv = cv3, n_jobs=1))
acc3

0.25

### Standardization

In [41]:
cv4 = KFold(n_splits = predictor2_cent.shape[0], shuffle = True, random_state = 1)

In [47]:
acc4 = np.mean(cross_val_score(forest4, predictor2_cent, target2, scoring = "accuracy", cv = cv4, n_jobs=1))
acc4

0.26973684210526316

# <span style="color:red">Random forest para las derivadas</span>

In [48]:
path_classes3 = "/home/franklinsierra/Documentos/Biomedica/Polipos/notebooks/mix_der_img/"
classes_names3 = os.listdir(path_classes3)
print (classes_names)
dic_img ={}
#numero de arboles para el random forest
num_trees = 100
#bins para el histograma
bins = 8
test_size = 0.2
#semilla para reproducir los mismos resultados
seed = 9

for i, name_class in enumerate(classes_names3):
    dic_img[classes_names3[i]] = np.array(os.listdir(path_classes3+classes_names3[i]))
    print ("class: ", name_class, "total ", len(dic_img[classes_names3[i]]))

['adenomaWl', 'hyperplasWl', 'hyperplasNbi', 'serratedWl', 'serratedNbi', 'adenomaNbi']
class:  hiperplasNbi total  21
class:  adenomaWl total  40
class:  serratedWl total  15
class:  serratedNbi total  15
class:  hiperplasWl total  21
class:  adenomaNbi total  40


In [49]:

predictor3 = []
target3 = []
resize_value = 100
for i, classes in enumerate(classes_names3):
    print((i+1), "clase: ", classes, " target: ", (i+1), " total: ", len(dic_img[classes]))
    for img_name in dic_img[classes]:
        path_image = path_classes3+classes+'/'+img_name
        img = cv.resize(cv.imread(path_image,0),(100,100))*1.
        flatt_img = img.reshape(resize_value*resize_value)        
        predictor3.append(flatt_img)
        target3.append(i+1)
predictor3 = np.r_[predictor3]        
target3 = np.r_[target3]

1 clase:  hiperplasNbi  target:  1  total:  21
2 clase:  adenomaWl  target:  2  total:  40
3 clase:  serratedWl  target:  3  total:  15
4 clase:  serratedNbi  target:  4  total:  15
5 clase:  hiperplasWl  target:  5  total:  21
6 clase:  adenomaNbi  target:  6  total:  40


### Normalizando

In [50]:
predictor3_norm = preprocessing.normalize(predictor3)

In [51]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train3, X_test3, y_train3, y_test3 = train_test_split(predictor3_norm,target3,test_size=test_size)
print (y_test3)

[5 5 2 6 1 2 2 3 1 5 5 4 6 1 5 2 5 4 6 6 6 1 3 2 3 2 2 1 2 4 1]


In [52]:
forest5 = RandomForestClassifier(n_jobs=2, oob_score=True, n_estimators=num_trees)
forest5.fit(X_train3,y_train3)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [53]:
forest5.score(X_test3,y_test3)

0.2903225806451613

### Standardization

In [55]:
predictor3_cent = preprocessing.scale(predictor3)

In [57]:
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(predictor3_cent,target3,test_size=test_size)
print (y_test)

[6 1 3 3 6 3 6 2 3 1 2 2 1 4 6 3 1 2 1 4 3 3 1 5 2 6 6 6 6 2 4]


In [58]:
forest6 = RandomForestClassifier(n_jobs=2, oob_score=True, n_estimators=num_trees)
forest6.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [59]:
forest6.score(X_test, y_test)

0.25806451612903225

# LOOCV

### Normalizated

In [62]:
cv5 = KFold(n_splits = predictor3_norm.shape[0], shuffle = True, random_state = 1)

In [63]:
acc5 = np.mean(cross_val_score(forest5, predictor3_norm, target3, scoring = "accuracy", cv = cv5, n_jobs=1))
acc5

0.3223684210526316

### Standardization

In [64]:
cv6 = KFold(n_splits = predictor3_cent.shape[0], shuffle = True, random_state = 1)

In [65]:
acc6 = np.mean(cross_val_score(forest6, predictor3_norm, target3, scoring = "accuracy", cv = cv6, n_jobs=1))
acc6

0.27631578947368424